In [10]:
import openpyxl
import os
import re

def read_data(sheet):
    # Find the last row with data in column A
    last_row = sheet.max_row
    for row in range(last_row, 2, -1):
        if sheet.cell(row=row, column=1).value is not None:
            last_row = row
            break

    data_range = f'A3:O{last_row}'
    data_range_values = sheet[data_range]
    data = [[cell.value for cell in row] for row in data_range_values]
    return data

def extract_date_from_filename(filename):
    # Modify the regular expression to match your filename pattern
    # For example, if your filename is like "StaffPerformanceOverviewCR06-09-23.xlsx"
    # and you want to extract "06-09-23", you can use r'\d{2}-\d{2}-\d{2}'
    pattern = r'\d{2}-\d{2}-\d{2}'
    match = re.search(pattern, filename)
    if match:
        return match.group()
    else:
        return None

def spo_adjustment(spoCR_path, spoNH_path):

    #Library Importation
    import openpyxl
    from openpyxl import load_workbook
    from openpyxl.styles import NamedStyle
    from collections import defaultdict
    from openpyxl.utils import get_column_letter
    import pandas as pd
    import numpy as np
    import re

    #Loading Workbooks
    wb1 = load_workbook (spoCR_path)
    wb2 = load_workbook(spoNH_path)
    ws1 = wb1.active
    ws2 = wb2.active

    #Appending Data To One Workbook
    for row in ws2.iter_rows(min_row=2, values_only=True): 
        ws1.append(row)

    date_match = re.search(r'(\d{2}-\d{2}-\d{2})', spoCR_path)
    if date_match:
        date = date_match.group(1)

    adjusted_filename = f'merged_spo_{date}.xlsx'

    # Define the folder path where you want to save the merged reports
    merged_SPO_folder = r'Reports Folder/Staff Performance Overview/Merged Reports SPO'

    # Construct the full path for the merged file
    merged_file_path = os.path.join(merged_SPO_folder, adjusted_filename)

    # Save the merged workbook to the specified absolute path
    wb1.save(merged_file_path)
    wb1.close()
    wb2.close()

    adjusted_workbook = openpyxl.load_workbook(merged_file_path)
    adjusted_sheet = adjusted_workbook.active

    def remove_labels(sheet):
        # Create a dictionary to keep track of row indices for each 'New' value
        new_rows = {}
    
        # Iterate through rows in reverse order to safely delete rows
        for row_index in range(sheet.max_row, 1, -1):
            cell_value = sheet.cell(row=row_index, column=3).value
            if cell_value == 'New':
                if cell_value not in new_rows:
                    new_rows[cell_value] = [row_index]
                else:
                    new_rows[cell_value].append(row_index)

        # Iterate through the dictionary and delete the second occurrence of 'New'
        for value, indices in new_rows.items():
            if len(indices) > 1:
                sheet.delete_rows(indices[0], 1)

    # Call the function to remove specific rows from the sheet
    remove_labels(adjusted_sheet)

    # Save the updated workbook after removing labels
    adjusted_workbook.save(merged_file_path)

    # Define the column letter you want to update (e.g., column A)
    column_letter = 'E'

    # Define the value to be replaced and its corresponding replacement
    old = "NA"
    new = "0"

    # Get the column index from the column letter
    column_index = ord(column_letter) - ord('A') + 1

    # Loop through the cells in the selected column and replace the old value with the new value
    for row in adjusted_sheet.iter_rows(min_row=3, min_col=column_index, max_col=column_index):
        cell = row[0]
        if cell.value == old:
            cell.value = new

    # Create named style for the whole number format
    whole_number_style = NamedStyle(name="whole_number", number_format="0")

    # Columns that need to be formatted as a whole number (example: columns A, B, and D)
    columns_to_format = ['B', 'C', 'D']  # Replace with the letters of the columns you want to format

    # Apply the named style to each cell in the selected columns
    for column_letter in columns_to_format:
        for cell in adjusted_sheet[column_letter]:
            cell.style = whole_number_style

    Decimal_Number_Style = NamedStyle(name="decimal", number_format="#,##0.00")

    columns_to_format = ['E', 'F', 'G', 'H','I','J','K','L','M','N','O']

    for column_letter in columns_to_format:
        for cell in adjusted_sheet[column_letter]:
            cell.style = Decimal_Number_Style

    adjusted_workbook.save(merged_file_path)
    adjusted_workbook.close()

    # Define custom functions for columns 14 and 15 based on columns 2, 3, and 4
    def calculate_AvgbillN(row):
    # Net Average Bill
        if row['Visits'] != 0:
            return row['TotalN'] / row['Visits']
        else:
            return 0  # Return a default value when 'Visits' is zero

    def calculate_AvgbillT(row):
    # Total Average Bill
        if row['Visits'] != 0:
            return row['TotalT'] / row['Visits']
        else:
            return 0  # Return a default value when 'Visits' is zero

    adjusted_workbook = openpyxl.load_workbook(merged_file_path)
    adjusted_worksheet = adjusted_workbook.active

    # Read data from the fourth_adjust_sheet
    data = read_data(adjusted_worksheet)  # Automatically reads data starting from A3

    # Create a DataFrame with all the data
    df = pd.DataFrame(data, columns=['EmployeeID', 'Visits', 'New', 'RQs', 'Rat', 'ServiceN', 'ServiceT', 'SeriesN', 
                                    'SeriesT', 'ProductsN', 'ProductsT', 'TotalN', 'TotalT', 'AvgbillN', 'AvgbillT'])

    # Get the union of EmployeeIDs
    all_employee_ids = set(df['EmployeeID'])

    # Create a DataFrame with all EmployeeIDs
    df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

   # Convert the 'EmployeeID' column to string in both dataframes
    df_all_employees['EmployeeID'] = df_all_employees['EmployeeID'].astype(str)
    df['EmployeeID'] = df['EmployeeID'].astype(str)
   
    # Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
    merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

    # Fill NaN values with 0
    merged_data.fillna(0, inplace=True)

    # Convert 'Data4', 'Data13', and 'Data14' columns to numeric type (ignore errors)
    merged_data['Rat'] = pd.to_numeric(merged_data['Rat'], errors='coerce')
    merged_data['AvgbillN'] = pd.to_numeric(merged_data['AvgbillN'], errors='coerce')
    merged_data['AvgbillT'] = pd.to_numeric(merged_data['AvgbillT'], errors='coerce')

    # Consolidate data based on EmployeeID and calculate the sum for each of the columns 2 to 13
    consolidated_data = merged_data.groupby('EmployeeID', as_index=False).agg({
        'Visits': 'sum',
        'New': 'sum',
        'RQs': 'sum',
        'Rat': 'sum',
        'ServiceN': 'sum',
        'ServiceT': 'sum',
        'SeriesN': 'sum',
        'SeriesT': 'sum',
        'ProductsN': 'sum',
        'ProductsT': 'sum',
        'TotalN': 'sum',
        'TotalT': 'sum',
        'AvgbillN': 'sum',
        'AvgbillT': 'sum'})

    # Calculate the average for Data4, Data13 columns for each individual employee
    consolidated_data['Rat'] = consolidated_data['Rat'] / 2

    # Calculate the values for columns 14 and 15 using the custom functions based on other columns
    consolidated_data['AvgbillN'] = consolidated_data.apply(calculate_AvgbillN, axis=1)
    consolidated_data['AvgbillT'] = consolidated_data.apply(calculate_AvgbillT, axis=1)

    # Calculate the column letters for the new columns
    conditional_column_letter = get_column_letter(adjusted_worksheet.max_column + 1)

    # Add new column headers
    conditional_column_header = "Conditional Column"
    adjusted_worksheet[conditional_column_letter + '1'] = conditional_column_header

    # Create a dictionary to store the count of employee occurrences
    employee_count = {}

    # Iterate through the rows (starting from the second row since the first row is headers)
    for row in range(2, adjusted_worksheet.max_row + 1):
        employee_cell = adjusted_worksheet.cell(row=row, column=1)  # Assuming employee names are in column 1 (A)
        employee_name = employee_cell.value

        if not employee_name:
            continue

        # Count the occurrences of each employee name
        if employee_name in employee_count:
            employee_count[employee_name] += 1
        else:
            employee_count[employee_name] = 1

    # Populate the new conditional column based on employee count
    for row in range(2, adjusted_worksheet.max_row + 1):
        employee_name = adjusted_worksheet.cell(row=row, column=1).value
        conditional_value = 2 if employee_count.get(employee_name, 0) > 1 else 1
        adjusted_worksheet.cell(row=row, column=adjusted_worksheet.max_column, value=conditional_value)
    
    # Create a list to store rows to be removed
    rows_to_remove = []

    # Identify rows with a conditional value of 2 and mark them for removal
    for row in range(2, adjusted_worksheet.max_row + 1):
        conditional_value = adjusted_worksheet.cell(row=row, column=adjusted_worksheet.max_column).value
        if conditional_value == 2:
            rows_to_remove.append(row)

    # Remove the marked rows in reverse order to avoid shifting issues
    for row in reversed(rows_to_remove):
        adjusted_worksheet.delete_rows(row)

     # Keep only the first two rows as headers
    rows_to_keep = [1, 2]
    rows_to_delete = [row for row in range(2, adjusted_worksheet.max_row + 1) if row not in rows_to_keep]   

    # Remove the non-header rows in reverse order to avoid shifting issues
    for row in reversed(rows_to_delete):
        adjusted_worksheet.delete_rows(row)

    # Delete the conditional column
    adjusted_worksheet.delete_cols(adjusted_worksheet.max_column)

    # Determine the last row of data in the worksheet
    last_row = adjusted_worksheet.max_row

    # Put the consolidated data back into the Excel ws
    for row_index, row_data in enumerate(consolidated_data.values):
        for col_index, cell_value in enumerate(row_data):
            cell = adjusted_worksheet.cell(row=last_row + 1 + row_index, column=col_index + 1)
            cell.value = cell_value

    # Save the updated workbook
    adjusted_workbook.save(merged_file_path)

# Call spo_adjustment for each batch of reports
if __name__ == '__main__':
# List of report batches (each batch is a list of report paths)
    reports_folder = 'Reports Folder/Staff Performance Overview/Individual Reports'
    report_files = os.listdir(reports_folder)

    # Group report files by their extracted dates
    date_to_reports = {}
    for filename in report_files:
        date = extract_date_from_filename(filename)
        if date:
            if date in date_to_reports:
                date_to_reports[date].append(filename)
            else:
                date_to_reports[date] = [filename]

    # Create report batches based on grouped dates
    report_batches = []
    for date, files in date_to_reports.items():
        batch_size = 2  # You can adjust this according to your needs
        for i in range(0, len(files), batch_size):
            batch = files[i:i+batch_size]
            report_batches.append(batch)

    for batch in report_batches:
        spoCR_path = os.path.join(reports_folder, batch[0])
        spoNH_path = os.path.join(reports_folder, batch[1])
        spo_adjustment(spoCR_path, spoNH_path)
    
